In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
from itertools import product
from scipy.stats import friedmanchisquare
import scikit_posthocs as sp
import networkx as nx



current_dir = os.getcwd()
parallel_folder = os.path.abspath(os.path.join(current_dir, '../Fit_and_Predict'))
sys.path.append(parallel_folder)

from config import get_model_ds_kwargs

/Users/leif/miniforge3/envs/hnam1/lib/python3.10/site-packages/pytorch_forecasting/models/base_model.py:30: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
for DATASET in ['Walmart','Favorita']:
    DATA = DATASET.lower()
    causals = get_model_ds_kwargs('HNAM', DATASET)['causal']
    scaling = pd.read_pickle(f'../Processed/{DATASET}/{DATASET.lower()}_scaling.pkl')
    odata = pd.read_pickle(f'../Processed/{DATASET}/{DATASET.lower()}_data.pkl')
    models = [model.split('.')[0] for model in os.listdir(f'{DATASET}') if model.endswith('.pkl') and not any(check in model for check in ['all','insample'])]
    data = pd.DataFrame()
    for model in models:
        if model == 'hnam_r':
            print('kicked ', model)
            continue
        print(model)
        data_temp = pd.read_pickle(f'{DATASET}/{model}.pkl')[['time_series', 'time_idx', 'pred_idx', 'h', 'true', 'pred', 'model']]
        if model == 'hnam_rhir':
            data_temp['model'] = 'HNAM_R'
        data = pd.concat([data, data_temp], axis=0)
    data = data[data.time_idx.isin(data.query('model == "Prophet"')['time_idx'].unique())]
    data = data.merge(odata[['time_idx','time_series','interpolated']],left_on=['pred_idx','time_series'],right_on=['time_idx','time_series'],how='left').drop(columns=['time_idx_y']).rename(columns={'time_idx_x':'time_idx'})

    naive1day = odata.pivot_table(index='time_idx',columns='time_series',values='sales').shift(1).melt(ignore_index=False,value_name='naive').dropna()
    odata = odata.merge(naive1day,left_on=['time_idx','time_series'],right_on=['time_idx','time_series'],how='left')

    data.groupby('model').nunique()
    odata['naive_error'] = (odata['sales'] - odata['naive']).abs()
    scaling_factors = odata.groupby(['time_series'])['naive_error'].mean()

    data['pred'] = data['pred'].clip(0)
    data['scaling_factor'] = data['time_series'].map(scaling_factors)
    data['mean'] = data['time_series'].map(scaling['mean'])
    data['std'] = data['time_series'].map(scaling['std'])
    data['pred_scaled'] = (data['pred']  - data['mean'])  /  data['std']
    data['true_scaled'] = (data['true']  - data['mean']) /  data['std']
    data['smape'] = 2 * np.abs(data['true'] - data['pred']) / (np.abs(data['true']) + np.abs(data['pred']))
    data['mape'] = np.abs(data['true'] - data['pred']) / np.abs(data['true'])
    data['smae'] = np.abs(data['true_scaled'] - data['pred_scaled'])
    data['srmse'] = ((data['true_scaled'] - data['pred_scaled']) ** 2)
    data['rmsse'] = (((data['true'] - data['pred']) / data['scaling_factor']) ** 2)
    errors = data[~data.interpolated.astype(bool)].groupby('model').agg({'srmse': ['mean','median'],'rmsse':['mean','median']}).sort_values(('srmse','mean')).round(3)
    errors['srmse'] = errors['srmse'] ** 0.5
    errors['rmsse'] = errors['rmsse'] ** 0.5
    errors = errors.applymap(lambda x: f'{x:.3f}')
    error_metrics = ['rmsse', 'srmse']
    model_performance = data[~data['interpolated'].astype(bool)].groupby('model')[error_metrics].agg(['mean', 'median'])
    model_performance = model_performance ** 0.5
    model_performance.columns = pd.MultiIndex.from_tuples(list(product(['RMSSE', 'sRMSE'], ['$\\bar{x}$', '$\\tilde{x}$'])))
    model_performance = model_performance.loc[["HNAM","TFT","ETS","ETSX","ARIMA","Prophet","Lasso"]]

    ranks = data.groupby(['time_series', 'model'])['srmse'].mean().unstack().rank(axis=1)
    rank1 = ranks.eq(1).mean()
    rank2 = ranks.eq(2).mean()

    model_performance[('Rank Freq.', '1\\textsuperscript{st}')] = rank1
    model_performance[('Rank Freq.', '2\\textsuperscript{nd}')] = rank2
    model_performance = model_performance.round(3)

    model_performance.index = [model.upper() for model in model_performance.index]

    numeric_model_performance = model_performance.copy()

    mean_srmse = data[~data['interpolated'].astype(bool)].groupby(['time_series', 'model'])['srmse'].mean().reset_index()
    srmse_pivot = mean_srmse.pivot(index='time_series', columns='model', values='srmse').dropna(axis=1)
    srmse_values = srmse_pivot.values

    statistic, p_value = friedmanchisquare(*srmse_values.T)
    print(f'Friedman test statistic: {statistic}, p-value: {p_value}')

    nemenyi_results = sp.posthoc_nemenyi_friedman(srmse_values)
    nemenyi_results.columns = srmse_pivot.columns
    nemenyi_results.index = srmse_pivot.columns

    alpha = 0.05 
    model_names = nemenyi_results.columns.tolist()
    G = nx.Graph()
    G.add_nodes_from(model_names)
    for i in range(len(model_names)):
        for j in range(i + 1, len(model_names)):
            model_i = model_names[i]
            model_j = model_names[j]
            p_val = nemenyi_results.loc[model_i, model_j]
            if p_val > alpha:
                G.add_edge(model_i, model_j)

    groups = list(nx.connected_components(G))

    group_performance = []
    for group in groups:
        group_models_upper = [model.upper() for model in group]
        group_srmse_means = numeric_model_performance.loc[group_models_upper, ('sRMSE', '$\\bar{x}$')].astype(float)
        group_mean_performance = group_srmse_means.mean()
        group_performance.append((group, group_mean_performance))

    group_performance.sort(key=lambda x: x[1])  # Ascending order

    group_labels = {}
    for group_num, (group, _) in enumerate(group_performance, start=1):
        for model in group:
            group_labels[model.upper()] = group_num
    new_index = []
    for model in model_performance.index:
        group_num = group_labels.get(model, '')
        new_model_name = f'{model}$^{{{group_num}}}$' if group_num else model
        new_index.append(new_model_name)
    model_performance.index = new_index
    numeric_model_performance.index = new_index 

    for column in model_performance.columns[:-2]:
        idxmin = numeric_model_performance[column].astype(float).idxmin()

        model_performance[column] = model_performance[column].apply(lambda x: f'{float(x):.3f}')
        model_performance.loc[idxmin, column] = f'\\textbf{{{model_performance.loc[idxmin, column]}}}'

    for column in model_performance.columns[-2:]:
        idxmax = numeric_model_performance[column].astype(float).idxmax()
        model_performance[column] = model_performance[column].apply(lambda x: f'{float(x):.1%}')
        model_performance.loc[idxmax, column] = f'\\textbf{{{model_performance.loc[idxmax, column]}}}'

    def custom_format(x):
        if isinstance(x, float):
            if abs(x) >= 1e2: 
                return f'{x:.1e}'  
            else:
                return f'{x:.3f}' 
        return x

    # Generate LaTeX table
    latex_string = model_performance.to_latex(
        caption=f'Mean and median error metrics in {DATASET.capitalize()}',
        label=f'tab:metrics_median_{DATASET.lower()}',
        position='h',
        column_format='rcccccc',
        multicolumn_format='c',
        formatters=[custom_format] * model_performance.shape[1],
    )

    latex_string = latex_string.replace('\\$','$')
    latex_string = latex_string.replace('\\{x\\','{x')
    latex_string = latex_string.replace('\\textbackslash ','\\')
    latex_string = latex_string.replace('\\{nd\\}','{nd}')
    latex_string = latex_string.replace('\\{st\\}','{st}')
    latex_string = latex_string.replace('%','\%')
    latex_string = latex_string.replace('ARIMA','SARIMAX')

    print(DATASET,' below')
    display(errors)
    display(model_performance)
    print(latex_string)
    print(DATASET,' above\n\n')

tft
etsx
prophet
lasso
kicked  hnam_r
ets
arima
hnam


/var/folders/0f/01zw64517cqdsbtf53d86j7h0000gn/T/ipykernel_22016/1691880104.py:41: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  errors = errors.applymap(lambda x: f'{x:.3f}')


Friedman test statistic: 1198.8622816032894, p-value: 8.438415283858842e-256
Walmart  below


srmse         rmsse       
          mean median   mean median
model                              
HNAM     0.853  0.505  1.050  0.624
TFT      0.860  0.527  1.064  0.653
ETS      0.881  0.525  1.093  0.648
Lasso    0.928  0.559  1.141  0.694
ETSX     0.935  0.553  1.155  0.683
ARIMA    0.980  0.577  1.207  0.713
Prophet  1.010  0.584  1.256  0.720

RMSSE                           sRMSE                  \
                    $\bar{x}$     $\tilde{x}$       $\bar{x}$     $\tilde{x}$   
HNAM$^{1}$     \textbf{1.050}  \textbf{0.625}  \textbf{0.853}  \textbf{0.505}   
TFT$^{1}$               1.064           0.653           0.860           0.528   
ETS$^{1}$               1.093           0.648           0.882           0.526   
ETSX$^{2}$              1.155           0.683           0.935           0.553   
ARIMA$^{3}$             1.207           0.713           0.980           0.577   
PROPHET$^{3}$           1.256           0.720           1.010           0.584   
LASSO$^{2}$             1.140           0.694           0.928           0.559   

                         Rank Freq.                        
              1\textsuperscript{st} 2\textsuperscript{nd}  
HNAM$^{1}$                    34.3%        \textbf{38.4%}  
TFT$^{1}$            \textbf{36.9%}                 23.7%  
ETS$^{1}$                     18.2%                 24.0%  
ETSX$^{2}$                     1.2%                  2.4%  
ARIMA$^{3}$                    0.7%                  1.4%  
PROPHET$^{3}$                  4.3%                  3.8%  
LASSO$^{2}$                    4.3%                  6.2%

...

/var/folders/0f/01zw64517cqdsbtf53d86j7h0000gn/T/ipykernel_22016/1691880104.py:41: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  errors = errors.applymap(lambda x: f'{x:.3f}')


Friedman test statistic: 1399.9557441323366, p-value: 2.4765791530864645e-299
Favorita  below


srmse         rmsse       
          mean median   mean median
model                              
TFT      0.829  0.396  1.119  0.456
HNAM     0.846  0.407  1.140  0.470
Prophet  0.896  0.438  1.216  0.503
ETS      0.897  0.434  1.262  0.496
ETSX     0.903  0.437  1.267  0.501
ARIMA    0.944  0.482  1.245  0.554
Lasso    0.972  0.497  1.300  0.574

RMSSE                           sRMSE                  \
                    $\bar{x}$     $\tilde{x}$       $\bar{x}$     $\tilde{x}$   
HNAM$^{1}$              1.140           0.470           0.846           0.407   
TFT$^{1}$      \textbf{1.119}  \textbf{0.457}  \textbf{0.829}  \textbf{0.397}   
ETS$^{2}$               1.262           0.496           0.897           0.433   
ETSX$^{3}$              1.267           0.501           0.903           0.437   
ARIMA$^{4}$             1.245           0.554           0.944           0.482   
PROPHET$^{2}$           1.216           0.503           0.895           0.439   
LASSO$^{5}$             1.300           0.574           0.972           0.497   

                         Rank Freq.                        
              1\textsuperscript{st} 2\textsuperscript{nd}  
HNAM$^{1}$                    21.2%        \textbf{30.4%}  
TFT$^{1}$            \textbf{43.6%}                 23.3%  
ETS$^{2}$                     12.3%                 14.9%  
ETSX$^{3}$                     4.7%                  9.8%  
ARIMA$^{4}$                    2.6%                  2.7%  
PROPHET$^{2}$                 14.6%                 17.1%  
LASSO$^{5}$                    0.9%                  1.7%

\begin{table}[h]
\caption{Mean and median error metrics in Favorita}
\label{tab:metrics_median_favorita}
\begin{tabular}{rcccccc}
\toprule
 & \multicolumn{2}{c}{RMSSE} & \multicolumn{2}{c}{sRMSE} & \multicolumn{2}{c}{Rank Freq.} \\
 & $\bar{x}$ & $\tilde{x}$ & $\bar{x}$ & $\tilde{x}$ & 1\textsuperscript{st} & 2\textsuperscript{nd} \\
\midrule
HNAM$^{1}$ & 1.140 & 0.470 & 0.846 & 0.407 & 21.2\% & \textbf{30.4\%} \\
TFT$^{1}$ & \textbf{1.119} & \textbf{0.457} & \textbf{0.829} & \textbf{0.397} & \textbf{43.6\%} & 23.3\% \\
ETS$^{2}$ & 1.262 & 0.496 & 0.897 & 0.433 & 12.3\% & 14.9\% \\
ETSX$^{3}$ & 1.267 & 0.501 & 0.903 & 0.437 & 4.7\% & 9.8\% \\
SARIMAX$^{4}$ & 1.245 & 0.554 & 0.944 & 0.482 & 2.6\% & 2.7\% \\
PROPHET$^{2}$ & 1.216 & 0.503 & 0.895 & 0.439 & 14.6\% & 17.1\% \\
LASSO$^{5}$ & 1.300 & 0.574 & 0.972 & 0.497 & 0.9\% & 1.7\% \\
\bottomrule
\end{tabular}
\end{table}

Favorita  above


